## LightGBM

Abrimos el dataset en el cual ya hemos trabajado las nuevas variables y caracteristicas.<br><br>
NOTAS:<br>
- algunas no fraude eliminar
- generar
- eliminar y aumentar
- matriz de confusión (clase objetivo) (clase 2)

In [65]:
#!pip install lightgbm
#!pip install --upgrade lightgbm
#!pip install category_encoders
#!pip install geopy

In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import logging
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt

In [67]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [68]:
data = pd.read_csv('final_dataset.csv')
data

,trans_date_trans_time,cc_num,merchant,category,amt,lat,long,trans_num,unix_time,merch_lat,...,yearly_txn_count_by_cat,season,seasonal_txn_count_by_cat,time_diff_hours,transaction_speed,amt_to_daily_avg,amt_to_monthly_avg,txn_count_last_hour,new_merchant,amt_to_annual_avg
0,2019-01-03 01:39:04,180011453250192,"fraud_Huel, Hammes and Witting",grocery_pos,90.37,41.2153,-90.9879,96a9fd4073d93d59eca9622dc21802c4,1325554744,41.810204,...,1,Winter,1,NaN,0.000000,0.225391,0.804453,1.0,True,0.925752
1,2019-01-03 05:51:22,180011453250192,fraud_Huel-Langworth,misc_net,108.09,41.2153,-90.9879,04cf40b2f392f3932b2532a92f1ef538,1325569882,41.016419,...,1,Winter,1,4.0,11.951459,0.269586,0.962193,1.0,True,1.107276
2,2019-01-03 14:48:47,180011453250192,fraud_Lemke and Sons,travel,447.52,41.2153,-90.9879,90d74d1a15d29a250fedbabe6721a483,1325602127,41.734914,...,1,Winter,1,8.0,8.712667,1.116156,3.983721,1.0,True,4.584404
3,2019-01-04 10:39:10,180011453250192,"fraud_Christiansen, Goyette and Schamberger",gas_transport,73.17,41.2153,-90.9879,52f1b4392482195579c35df07514962a,1325673550,40.744024,...,1,Winter,1,19.0,3.096099,0.182493,0.651343,1.0,True,0.749555
4,2019-01-04 12:48:21,180011453250192,fraud_Ruecker-Mayert,kids_pets,47.55,41.2153,-90.9879,63954f859ede83d209aece226970441b,1325681301,41.462258,...,1,Winter,1,2.0,41.738353,0.118594,0.423279,1.0,True,0.487103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 14:49:51,676372984911,fraud_Kris-Padberg,shopping_pos,6.13,39.7562,-98.4462,e327aa916b4317b8baf9d181855bf9df,1388501391,39.553152,...,145,Winter,96,3.0,24.693907,0.019995,0.087796,1.0,False,0.094527
1852390,2020-12-31 16:17:19,676372984911,fraud_Schiller Ltd,personal_care,62.99,39.7562,-98.4462,2d10acccfe94d134bdac9c41c614cb00,1388506639,40.024226,...,145,Winter,95,1.0,34.733267,0.205466,0.902169,1.0,False,0.971327
1852391,2020-12-31 20:21:23,676372984911,fraud_Pollich LLC,home,28.11,39.7562,-98.4462,24ffee6f547efdbffcf471498e55e0eb,1388521283,40.412189,...,195,Winter,88,4.0,21.983892,0.091692,0.402603,1.0,False,0.433466
1852392,2020-12-31 20:58:25,676372984911,"fraud_Hayes, Marquardt and Dibbert",misc_net,211.32,39.7562,-98.4462,898279ea0067e89917bdf288f96d6834,1388523505,40.240147,...,91,Winter,36,0.0,inf,0.689301,3.026613,2.0,False,3.258625


In [69]:
data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'lat',
       'long', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud',
       'amt_month', 'amt_year', 'amt_month_shopping_net_spend',
       'count_month_shopping_net', 'first_time_at_merchant',
       'time_since_last_txn', 'avg_daily_amt', 'hour',
       'avg_hourly_transactions', 'prev_lat', 'prev_long',
       'distance_from_last_txn', 'month_year', 'monthly_spending_variation',
       'weekday', 'is_holiday', 'first_txn_time', 'days_since_first_txn',
       'txn_count_last_24h', 'avg_amt_last_24h', 'std_amt_last_24h',
       'online_txn_ratio', 'trans_date', 'monthly_txn_count_by_cat',
       'trans_year', 'yearly_txn_count_by_cat', 'season',
       'seasonal_txn_count_by_cat', 'time_diff_hours', 'transaction_speed',
       'amt_to_daily_avg', 'amt_to_monthly_avg', 'txn_count_last_hour',
       'new_merchant', 'amt_to_annual_avg'],
      dtype='object')

Vamos a convertir aquellas columnas que no sean datos numericos con un encoder

In [70]:
columns_to_encode = ['merchant', 'category', 'trans_num', 'cc_num', 'season']

with open('label_encodings.txt', 'w') as file:
    for col in columns_to_encode:
        encoder = LabelEncoder()
        data[col + '_encoded'] = encoder.fit_transform(data[col])
        mappings = {index: label for index, label in enumerate(encoder.classes_)}
        file.write(f"{col} mappings:\n")
        for key, value in mappings.items():
            file.write(f"{key}: {value}\n")
        file.write("\n")

data.drop(columns=columns_to_encode, axis=1, inplace=True)

In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 47 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   trans_date_trans_time         object 
 1   amt                           float64
 2   lat                           float64
 3   long                          float64
 4   unix_time                     int64  
 5   merch_lat                     float64
 6   merch_long                    float64
 7   is_fraud                      int64  
 8   amt_month                     float64
 9   amt_year                      float64
 10  amt_month_shopping_net_spend  float64
 11  count_month_shopping_net      float64
 12  first_time_at_merchant        bool   
 13  time_since_last_txn           float64
 14  avg_daily_amt                 float64
 15  hour                          int64  
 16  avg_hourly_transactions       float64
 17  prev_lat                      float64
 18  prev_long             

In [72]:
data.columns

Index(['trans_date_trans_time', 'amt', 'lat', 'long', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud', 'amt_month', 'amt_year',
       'amt_month_shopping_net_spend', 'count_month_shopping_net',
       'first_time_at_merchant', 'time_since_last_txn', 'avg_daily_amt',
       'hour', 'avg_hourly_transactions', 'prev_lat', 'prev_long',
       'distance_from_last_txn', 'month_year', 'monthly_spending_variation',
       'weekday', 'is_holiday', 'first_txn_time', 'days_since_first_txn',
       'txn_count_last_24h', 'avg_amt_last_24h', 'std_amt_last_24h',
       'online_txn_ratio', 'trans_date', 'monthly_txn_count_by_cat',
       'trans_year', 'yearly_txn_count_by_cat', 'seasonal_txn_count_by_cat',
       'time_diff_hours', 'transaction_speed', 'amt_to_daily_avg',
       'amt_to_monthly_avg', 'txn_count_last_hour', 'new_merchant',
       'amt_to_annual_avg', 'merchant_encoded', 'category_encoded',
       'trans_num_encoded', 'cc_num_encoded', 'season_encoded'],
      dtype='object')

Observamos detalladamente por aquellas columnas que no son numéricas que pueden necesitas codificación:

In [73]:
non_numeric_columns = data.select_dtypes(include=['object']).columns

print("Columnas no numéricas que podrían necesitar codificación:")
non_numeric_columns

Columnas no numéricas que podrían necesitar codificación:


Index(['trans_date_trans_time', 'month_year', 'first_txn_time', 'trans_date'], dtype='object')

Ahora vamos a generar los sub datasets donde la información de cada mes será agrupada en un solo set para poder hacer entrenamiento incremental por mes.

In [74]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data.set_index('trans_date_trans_time', inplace=True)

In [75]:
monthly_groups = data.groupby(pd.Grouper(freq='M'))
monthly_datasets = {}
for name, group in monthly_groups:
    print(f"Datos para el mes: {name}")
    monthly_datasets[name] = group
    print(group.head())
    print("\n-------------------------------------------\n")


Datos para el mes: 2019-01-31 00:00:00
                          amt      lat      long   unix_time  merch_lat  \
trans_date_trans_time                                                     
2019-01-01 00:00:18      4.97  36.0788  -81.1781  1325376018  36.011293   
2019-01-01 00:00:44    107.23  48.8878 -118.2105  1325376044  49.159047   
2019-01-01 00:00:51    220.11  42.1808 -112.2620  1325376051  43.150704   
2019-01-01 00:01:16     45.00  46.2306 -112.1138  1325376076  47.034331   
2019-01-01 00:03:06     41.96  38.4207  -79.4629  1325376186  38.674999   

                       merch_long  is_fraud  amt_month  amt_year  \
trans_date_trans_time                                              
2019-01-01 00:00:18    -82.048315         0       4.97      4.97   
2019-01-01 00:00:44   -118.186462         0     107.23    107.23   
2019-01-01 00:00:51   -112.154481         0     220.11    220.11   
2019-01-01 00:01:16   -112.561071         0      45.00     45.00   
2019-01-01 00:03:06    -78.

In [1]:
# reset index y drop = true para reconocer columna

NameError: name 'data' is not defined

In [76]:
columns_to_remove = ['trans_date_trans_time', 'month_year', 'first_txn_time', 'trans_date']
for month, dataset in monthly_datasets.items():
    existing_columns_to_remove = [col for col in columns_to_remove if col in dataset.columns]
    if existing_columns_to_remove:
        dataset.drop(columns=existing_columns_to_remove, inplace=True)
    monthly_datasets[month] = dataset

C:\Users\marco\AppData\Local\Temp\ipykernel_15516\3015772301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(columns=existing_columns_to_remove, inplace=True)
C:\Users\marco\AppData\Local\Temp\ipykernel_15516\3015772301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop(columns=existing_columns_to_remove, inplace=True)
C:\Users\marco\AppData\Local\Temp\ipykernel_15516\3015772301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [77]:
for month, dataset in monthly_datasets.items():
    valid_types = all(dtype in [np.number, np.bool_] for dtype in dataset.dtypes)
    if not valid_types:
        print(f"El dataset para {month.strftime('%Y-%m')} contiene tipos no válidos para LGBM.")
    else:
        print(f"El dataset para {month.strftime('%Y-%m')} está listo para LGBM.")


El dataset para 2019-01 contiene tipos no válidos para LGBM.
El dataset para 2019-02 contiene tipos no válidos para LGBM.
El dataset para 2019-03 contiene tipos no válidos para LGBM.
El dataset para 2019-04 contiene tipos no válidos para LGBM.
El dataset para 2019-05 contiene tipos no válidos para LGBM.
El dataset para 2019-06 contiene tipos no válidos para LGBM.
El dataset para 2019-07 contiene tipos no válidos para LGBM.
El dataset para 2019-08 contiene tipos no válidos para LGBM.
El dataset para 2019-09 contiene tipos no válidos para LGBM.
El dataset para 2019-10 contiene tipos no válidos para LGBM.
El dataset para 2019-11 contiene tipos no válidos para LGBM.
El dataset para 2019-12 contiene tipos no válidos para LGBM.
El dataset para 2020-01 contiene tipos no válidos para LGBM.
El dataset para 2020-02 contiene tipos no válidos para LGBM.
El dataset para 2020-03 contiene tipos no válidos para LGBM.
El dataset para 2020-04 contiene tipos no válidos para LGBM.
El dataset para 2020-05 

C:\Users\marco\AppData\Local\Temp\ipykernel_15516\2165270538.py:2: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  valid_types = all(dtype in [np.number, np.bool_] for dtype in dataset.dtypes)


In [78]:
# Data no es datasets de cada mes
for column in data.columns:
    if data[column].dtype == 'int64' or data[column].dtype == 'int32':
        data[column] = data[column].astype(float)
    elif data[column].dtype == 'bool':
        data[column] = data[column].astype(int)  # Convertir booleanos a enteros

print(data.dtypes)


amt                             float64
lat                             float64
long                            float64
unix_time                       float64
merch_lat                       float64
merch_long                      float64
is_fraud                        float64
amt_month                       float64
amt_year                        float64
amt_month_shopping_net_spend    float64
count_month_shopping_net        float64
first_time_at_merchant            int32
time_since_last_txn             float64
avg_daily_amt                   float64
hour                            float64
avg_hourly_transactions         float64
prev_lat                        float64
prev_long                       float64
distance_from_last_txn          float64
month_year                       object
monthly_spending_variation      float64
weekday                         float64
is_holiday                        int32
first_txn_time                   object
days_since_first_txn            float64


In [84]:
# Revisar los tipos de datos de cada dataset mensual
for month, dataset in monthly_datasets.items():
    print(f"Tipos de datos para el mes: {month.strftime('%Y-%m')}")
    print(dataset.dtypes)  # Mostrar los tipos de datos
    print("\n-------------------------------------------\n")

Tipos de datos para el mes: 2019-01
amt                             float64
lat                             float64
long                            float64
unix_time                         int64
merch_lat                       float64
merch_long                      float64
is_fraud                          int64
amt_month                       float64
amt_year                        float64
amt_month_shopping_net_spend    float64
count_month_shopping_net        float64
first_time_at_merchant             bool
time_since_last_txn             float64
avg_daily_amt                   float64
hour                              int64
avg_hourly_transactions         float64
prev_lat                        float64
prev_long                       float64
distance_from_last_txn          float64
monthly_spending_variation      float64
weekday                           int64
is_holiday                         bool
days_since_first_txn              int64
txn_count_last_24h              float64
avg_

In [79]:
for month, dataset in monthly_datasets.items():
    valid_types = all(dtype in [np.number, np.bool_] for dtype in dataset.dtypes)
    if not valid_types:
        print(f"El dataset para {month.strftime('%Y-%m')} contiene tipos no válidos para LGBM.")
    else:
        print(f"El dataset para {month.strftime('%Y-%m')} está listo para LGBM.")

El dataset para 2019-01 contiene tipos no válidos para LGBM.
El dataset para 2019-02 contiene tipos no válidos para LGBM.
El dataset para 2019-03 contiene tipos no válidos para LGBM.
El dataset para 2019-04 contiene tipos no válidos para LGBM.
El dataset para 2019-05 contiene tipos no válidos para LGBM.
El dataset para 2019-06 contiene tipos no válidos para LGBM.
El dataset para 2019-07 contiene tipos no válidos para LGBM.
El dataset para 2019-08 contiene tipos no válidos para LGBM.
El dataset para 2019-09 contiene tipos no válidos para LGBM.
El dataset para 2019-10 contiene tipos no válidos para LGBM.
El dataset para 2019-11 contiene tipos no válidos para LGBM.
El dataset para 2019-12 contiene tipos no válidos para LGBM.
El dataset para 2020-01 contiene tipos no válidos para LGBM.
El dataset para 2020-02 contiene tipos no válidos para LGBM.
El dataset para 2020-03 contiene tipos no válidos para LGBM.
El dataset para 2020-04 contiene tipos no válidos para LGBM.
El dataset para 2020-05 

C:\Users\marco\AppData\Local\Temp\ipykernel_15516\2808465799.py:2: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  valid_types = all(dtype in [np.number, np.bool_] for dtype in dataset.dtypes)


Procedemos a separar el dataset en 80% entrenamiento y 20% validación.

In [80]:
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [81]:
X_train.isnull().sum()

amt                                   0
lat                                   0
long                                  0
unix_time                             0
merch_lat                             0
merch_long                            0
amt_month                             0
amt_year                              0
amt_month_shopping_net_spend          0
count_month_shopping_net              0
first_time_at_merchant                0
time_since_last_txn                 603
avg_daily_amt                         0
hour                                  0
avg_hourly_transactions               0
prev_lat                            603
prev_long                           603
distance_from_last_txn                0
month_year                            0
monthly_spending_variation        31854
weekday                               0
is_holiday                            0
first_txn_time                        0
days_since_first_txn                  0
txn_count_last_24h                    0


In [82]:
X_train.describe()

,amt,lat,long,unix_time,merch_lat,merch_long,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,...,amt_to_daily_avg,amt_to_monthly_avg,txn_count_last_hour,new_merchant,amt_to_annual_avg,merchant_encoded,category_encoded,trans_num_encoded,cc_num_encoded,season_encoded
count,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,...,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06,1.111436e+06
mean,7.021377e+01,3.854654e+01,-9.023018e+01,1.358668e+09,3.854583e+01,-9.022981e+01,4.152791e+03,4.530028e+04,3.761505e+02,4.566486e+00,...,3.000151e-01,1.001114e+00,1.200110e+00,3.877866e-04,1.002024e+00,3.426389e+02,6.228754e+00,9.265608e+05,4.996167e+02,1.575599e+00
std,1.617274e+02,5.070867e+00,1.375339e+01,1.819090e+07,5.104951e+00,1.376507e+01,3.904934e+03,3.584776e+04,7.252580e+02,4.570191e+00,...,7.852057e-01,1.706102e+00,4.758421e-01,1.968849e-02,2.273992e+00,2.011720e+02,3.914724e+00,5.347360e+05,2.861838e+02,1.105049e+00
min,1.000000e+00,2.002710e+01,-1.656723e+02,1.325376e+09,1.902742e+01,-1.666716e+02,1.000000e+00,1.020000e+00,0.000000e+00,0.000000e+00,...,1.774139e-03,1.110568e-03,1.000000e+00,0.000000e+00,7.643465e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.650000e+00,3.466890e+01,-9.679800e+01,1.343037e+09,3.475044e+01,-9.690296e+01,1.345947e+03,1.733403e+04,9.030000e+00,1.000000e+00,...,4.456167e-02,1.675709e-01,1.000000e+00,0.000000e+00,1.557362e-01,1.650000e+02,3.000000e+00,4.635628e+05,2.510000e+02,1.000000e+00
50%,4.751000e+01,3.937160e+01,-8.747690e+01,1.357057e+09,3.937697e+01,-8.745271e+01,3.073385e+03,3.742817e+04,7.611000e+01,3.000000e+00,...,1.767692e-01,6.758996e-01,1.000000e+00,0.000000e+00,6.718552e-01,3.450000e+02,6.000000e+00,9.266480e+05,4.930000e+02,2.000000e+00
75%,8.312000e+01,4.194040e+01,-8.015800e+01,1.374565e+09,4.196239e+01,-8.023695e+01,5.738315e+03,6.475312e+04,4.256200e+02,7.000000e+00,...,3.518220e-01,1.257132e+00,1.000000e+00,0.000000e+00,1.193660e+00,5.140000e+02,1.000000e+01,1.389140e+06,7.470000e+02,3.000000e+00
max,2.739012e+04,6.669330e+01,-6.795030e+01,1.388534e+09,6.739702e+01,-6.695203e+01,4.326189e+04,2.190868e+05,1.204718e+04,4.800000e+01,...,1.846413e+02,1.297801e+02,9.000000e+00,1.000000e+00,2.948023e+02,6.920000e+02,1.300000e+01,1.852388e+06,9.980000e+02,3.000000e+00


### Entrenamiento con LGBM

In [83]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1
}
lgb_model = None

# Iterar sobre cada segmento mensual
for name, month_data in monthly_groups:
    print(f"Entrenando con datos de: {name}")

    X_train_month = month_data.drop(['is_fraud'], axis=1)
    y_train_month = month_data['is_fraud']
    train_data_month = lgb.Dataset(X_train_month, label=y_train_month)

    if lgb_model is None:
        lgb_model = lgb.train(params, train_data_month, num_boost_round=100)
    else:
        # Entrenamiento incremental
        lgb_model = lgb.train(params, train_data_month, init_model=lgb_model, num_boost_round=10)

Entrenando con datos de: 2019-01-31 00:00:00


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: month_year: object, first_txn_time: object, trans_date: object

In [ ]:
y_pred = (lgb_model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')